<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QTURJgbm1_Z_CQW3suD0tRP3mrNgqt70#scrollTo=oIWrv9IYY2OT)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

## 🛠️✨ **DSPy (Data Science Prompting)**  
DSPy is the framework for programming—rather than prompting—language models. It allows you to iterate fast on building modular AI systems and offers algorithms for optimizing their prompts and weights, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.


###Setup and Installation

Install Required Libraries


In [ ]:
pip install httpx==0.27.2 dspy faiss-cpu

###Configure OpenAI API

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
OPENAIKEY=os.getenv('OPENAI_API_KEY')


###Setting Up DSPY

In [ ]:
import dspy
lm = dspy.OpenAI(model='gpt-4o', api_key=OPENAIKEY, model_type='chat', max_tokens = 500)
dspy.settings.configure(lm=lm)

### 📂 Loading the Dataset
In this step, the HotPotQA dataset is loaded with specified sizes for training and development sets. The dataset is further processed to include 'question' inputs for each example, preparing it for training and evaluation tasks.







In [ ]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

### 🔍 Inspecting a Training Example

In [ ]:
train_example = trainset[0]
print(train_example)
print(f"Question: {train_example.question}")
print(f"Answer: {train_example.answer}")

### 🛠️ Defining the BasicQA Signature

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

### 🤖 Creating the BasicQABot Module

In [ ]:
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.Predict(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

### 🧠 Querying the QA Bot

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    qa_bot = BasicQABot()
    pred = qa_bot.forward("In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?")
    print(pred.answer)

##Retrieval-Augmented Generation (RAG)
quick example of basic question answering with and without retrieval-augmented generation (RAG) in DSPy.

###Configuring the DSPy environment.

In [ ]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

###Simple QA Chain

In [ ]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

###Inspect History


In [ ]:
dspy.inspect_history(n=1)

###Chain Of Thought Module

In [ ]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='The placement of curly braces on their own line depends on the coding style and conventions being followed. In some programming languages and style guides, such as the Allman style, opening curly braces are placed on a new line, which can enhance readability. In contrast, other styles, like K&R style, place the opening brace on the same line as the control statement, which can save vertical space. Ultimately, it is a matter of personal or team preference, and consistency within a codebase is key.',
    response="Curly braces can either appear on their own line or on the same line as the preceding statement, depending on the coding style you choose to follow. It's important to maintain consistency throughout your codebase."
)

###Manipulating Examples in DSPy.

###Let's load a dataset of questions and their (pretty long) gold answers.

In [ ]:
import ujson
from dspy.utils import download

# Download question--answer pairs from the RAG-QA Arena "Tech" dataset.
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

### Inspect one datapoint.


In [ ]:
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

#### Let's pick an `example` here from the data.


In [ ]:
data = [dspy.Example(**d).with_inputs('question') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

###Now, let's divide the data into: Train Set

In [ ]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)

(200, 300, 500)

###Evaluation in DSPy.

In [ ]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 why are my text messages coming up as maybe?

Gold Response: 	 This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". 

However, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.

Predicted Response: 	 Your text messages are showing up as "maybe" because the recipient's phone or messaging app is indicating that the sender is not recognized or saved in their contacts. This feature helps users identify messages from unknown numbers. To avoid this, you can ask the recipient to save your number in their contacts.

Semantic F1 Score: 0.00


###Inspect History

In [ ]:
dspy.inspect_history(n=1)

For evaluation, you could use the metric above in a simple loop and just average the score. But for nice parallelism and utilities, we can rely on dspy.Evaluate.


In [ ]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

##Math Reasoning

###Fetching The Dataset

In [ ]:
%pip install git+https://github.com/hendrycks/math.git

###Setting UP DSPY

In [28]:
import dspy

gpt4o_mini = dspy.LM('openai/gpt-4o-mini', max_tokens=2000)
gpt4o = dspy.LM('openai/gpt-4o', max_tokens=2000)
dspy.configure(lm=gpt4o_mini)  # we'll use gpt-4o-mini as the default LM, unless otherwise specified

###Load some data examples from the MATH benchmark.

In [ ]:
from dspy.datasets import MATH

dataset = MATH(subset='algebra')
print(len(dataset.train), len(dataset.dev))

####Inspect one example from the training set.

In [ ]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

###Chain of Thought Module

In [ ]:
module = dspy.ChainOfThought("question -> answer")
module(question=example.question)

###set up an evaluator for the zero-shot module

In [ ]:
THREADS = 24
kwargs = dict(num_threads=THREADS, display_progress=True, display_table=5)
evaluate = dspy.Evaluate(devset=dataset.dev, metric=dataset.metric, **kwargs)

evaluate(module)

###Optimize Module

In [ ]:
kwargs = dict(num_threads=THREADS, teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o_mini)
optimizer = dspy.MIPROv2(metric=dataset.metric, auto="medium", **kwargs)

kwargs = dict(requires_permission_to_run=False, max_bootstrapped_demos=4, max_labeled_demos=4)
optimized_module = optimizer.compile(module, trainset=dataset.train, **kwargs)

In [ ]:
evaluate(optimized_module)

###Inspect History

In [ ]:
dspy.inspect_history()